In [5]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from numba import njit

In [6]:
@njit(fastmath=True)
def Brownian_simulator(S, v, T, r, q, Number_of_simulation, steps,seed=None):
    if seed is not None:
        np.random.seed(seed)
    t = T / steps
    paths = np.zeros((Number_of_simulation, steps + 1))
    

    drift = (r - q - 0.5 * v**2) * t
    noise = np.sqrt(t) * v
    for i in range(Number_of_simulation):
        Z = np.random.standard_normal(steps)
        increments = drift + noise * Z
        brownian_path = np.cumsum(increments)
        paths[i, 1:] = brownian_path
    final_simulation = S * np.exp(paths)
    return final_simulation

class option:
    def __init__(self,Spot,Strike,vol,Time,rates,Dividend_yield):
        self.S=Spot
        self.K=Strike
        self.v=vol
        self.T=Time
        self.r=rates
        self.q=Dividend_yield
        if np.any(Spot<0) or vol<0 or Time<=0:
            raise ValueError('The values are not following the assumptions of the model ')




    def summary(self):
        print(f'you are pricing an option with parameters: S={self.S}, K={self.K}, vol={self.v}, T={self.T}, r={self.r}, q={self.q}')


class European_option(option):
    @property
    def d1(self):
        return (np.log(self.S/self.K)+(self.r-self.q+0.5*(self.v**2))*self.T)/(self.v*np.sqrt(self.T))
    
    @property
    def d2(self):
        return self.d1-self.v*np.sqrt(self.T)

        

    def Call(self):
        return self.S*np.exp(-self.q*self.T)*norm.cdf(self.d1,0,1)-self.K*np.exp(-self.r*self.T)*norm.cdf(self.d2,0,1)
    
    def Put(self):
        return -self.S*np.exp(-self.q*self.T)*norm.cdf(-self.d1,0,1)+self.K*np.exp(-self.r*self.T)*norm.cdf(-self.d2,0,1)
    
    def parity(self):
        C=self.Call()
        P=self.Put()
        if C+self.K*np.exp(-self.r*self.T)!= P+self.S*np.exp(-self.d*self.T):
            return("Arbitrage opportunity the call/put parity is not respected ")
        else:
            return("No arbitrage opportunity, the call/put parity is respected")

    
    def Delta(self,option_type):

        if str(option_type.capitalize())=='Call':
            return norm.cdf(self.d1,0,1)*np.exp(-self.q*self.T)
        
        elif str(option_type.capitalize())=='Put':
            return -norm.cdf(-self.d1,0,1)*np.exp(-self.q*self.T)
        
        else:
            raise ValueError('please select Call or Put')


    def delta_graph(self,start,end,option_type):

        intitial_price=self.S
        self.S=np.linspace(max(0.1,start),end,1000)
        plt.figure()
        plt.plot(self.S,self.Delta(str(option_type.capitalize())))
        plt.title('Evolution of Delta')
        plt.xlabel('Underlying asset price')
        plt.ylabel('Delta')
        plt.grid(True)
        self.S=intitial_price
        
    
    def Gamma(self):
        return (norm.pdf(self.d1,0,1)/(self.S*np.sqrt(self.T)*self.v))*np.exp(-self.d*self.T)
    

    def gamma_graph(self,start,end):

        intitial_price=self.S
        self.S=np.linspace(max(0.01,start),end,1000)
        plt.figure()
        plt.plot(self.S,self.Gamma())
        plt.title('Evolution of Gamma')
        plt.xlabel('Underlying asset price')
        plt.ylabel('Gamma')
        plt.grid(True)
        self.S=intitial_price
    

    def Vega(self):
        return self.S*np.exp(-self.q*self.T)*norm.pdf(self.d1,0,1)*np.sqrt(self.T)/100
    
    def vega_graph(self):
        initial_price=self.S
        self.S=np.linspace(0,self.S*3,300)
        plt.figure()
        plt.plot(self.S,self.Vega())
        plt.title('Evolution of Vega')
        plt.xlabel('Underlying asset price')
        plt.ylabel('Vega')
        plt.grid(True)
        self.S=initial_price


    def Theta(self,option_type):

        if str(option_type.capitalize())=='Call':
            return(-((self.S*np.exp(-self.q*self.T)*norm.pdf(self.d1,0,1)*self.v)/(2*np.sqrt(self.T)))+self.q*self.S*np.exp(-self.q*self.T)* norm.cdf(self.d1, 0, 1)-self.r*self.K*np.exp(-self.r*self.T)*norm.cdf(self.d2,0,1))/252
        
        elif str(option_type.capitalize())=='Put':
            return(-((self.S*np.exp(-self.q*self.T)*norm.pdf(self.d1,0,1)*self.v)/(2*np.sqrt(self.T)))-self.q*self.S*np.exp(-self.q*self.T)* norm.cdf(-self.d1, 0, 1)+self.r*self.K*np.exp(-self.r*self.T)*norm.cdf(-self.d2,0,1))/252
        
        else:
            raise ValueError('please select Call or Put')
        
    def theta_graph(self, end, option_type):
        intitial_price=self.S
        self.S=np.linspace(0,int(end),300)
        plt.figure()
        plt.plot(self.S,self.Theta(option_type.capitalize()))
        plt.title('Evolution of Theta')
        plt.xlabel('Underlying asset price')
        plt.ylabel('Theta')
        plt.grid(True)
        self.S=intitial_price


    def Rho(self, option_type):
        
        if str(option_type.capitalize())=='Call':
            return (self.K*self.T*np.exp(-self.r*self.T)*norm.cdf(self.d2,0,1))/100
        
        elif str(option_type.capitalize())=='Put':
            return (-self.K*self.T*np.exp(-self.r*self.T)*norm.cdf(-self.d2,0,1))/100
        
        else:
            raise ValueError('please select Call or Put')
        
    def rho_graph(self, option_type):
        intitial_price=self.S
        self.S=np.linspace(0,intitial_price*2,300)
        plt.figure()
        plt.plot(self.S,self.Rho(option_type.capitalize()))
        plt.title('Evolution of Rho')
        plt.xlabel('Underlying asset price')
        plt.ylabel('Rho')
        plt.grid(True)
        self.S=intitial_price



class American_option(option):
    def American_option_price(self,N,option_type):


        t=self.T/N
        u=np.exp(self.v*np.sqrt(t))
        d=1/u
        p=(np.exp((self.r-self.q)*t)-d)/(u-d)


        B=np.zeros(shape=[N+1,N+1])

        B[0,0]=self.S

        for i in range (N+1):
            for j in range(i+1):
                B[j,i]=self.S*(u**(i-j))*(d**(j))
        C=np.zeros(shape=[N+1,N+1])

        if option_type.capitalize() == 'Call':
            C[:, N] = np.maximum(B[:, N] - self.K, 0)
        elif option_type.capitalize() == 'Put':
            C[:, N] = np.maximum(self.K - B[:, N], 0)
        else:
            raise ValueError('Please select Call or Put')


        for i in range(N - 1, -1, -1):
            for j in range(i + 1):

                hold_value = np.exp(-self.r * t) * (p * C[j, i + 1] + (1 - p) * C[j + 1, i + 1])
            
            

                if option_type.capitalize() == 'Call':
                    exercise_value = max(B[j, i] - self.K, 0)
                else:
                    exercise_value = max(self.K - B[j, i], 0)
                C[j, i] = max(hold_value, exercise_value)
            
    
        return C[0,0]
    

class Binary_option(option):
    
    @property
    def d2(self):
        return self.d1-self.v*np.sqrt(self.T)

    def price(self,payoff,option_type):
        if str(option_type.capitalize())=='Call':
            return np.exp(-self.r*self.T)*norm.cdf(self.d2,0,1)*payoff
        elif str(option_type.capitalize())=='Put':
            return np.exp(-self.r*self.T)*norm.cdf(-self.d2,0,1)*payoff

class Asian_option(option):

    def Asian_option_price(self,Number_of_simulation, option_type,steps,seed=None):

        simulation=Brownian_simulator(self.S,self.v,self.T,self.r,self.q,Number_of_simulation,steps,seed)
        mean=np.mean(simulation,axis=1)

        if option_type.capitalize()=='Call':
            asian_payoff=np.maximum(mean-self.K,0)
            asian_payoff=np.mean(asian_payoff)*np.exp(-self.r*self.T)
            return asian_payoff
        if option_type.capitalize()=='Put':
            asian_payoff=np.maximum(self.K-mean,0)
            asian_payoff=np.mean(asian_payoff)*np.exp(-self.r*self.T)
            return asian_payoff
        else:
            raise ValueError('please select Call or Put')
        
        
    def Asian_option_delta(self,e,option_type,seed=None):
        first_option=Asian_option(self.S+e,self.K,self.v,self.T,self.r,self.q)
        second_option=Asian_option(self.S-e,self.K,self.v,self.T,self.r,self.q)
        delta=(first_option.Asian_option_price(1000000,option_type,252,seed)-second_option.Asian_option_price(1000000,option_type,252,seed))/(2*e)
        return delta
    
    def Asian_option_vega(self,e,option_type,seed=None):
        first_option=Asian_option(self.S,self.K,self.v+e,self.T,self.r,self.q)
        second_option=Asian_option(self.S,self.K,self.v-e,self.T,self.r,self.q)
        vega=(first_option.Asian_option_price(1000000,option_type,252,seed)-second_option.Asian_option_price(1000000,option_type,252,seed))/(2*e)
        return vega/100
    
    def Asian_option_rho(self,e,option_type,seed=None):
        first_option=Asian_option(self.S,self.K,self.v,self.T,self.r+e,self.q)
        second_option=Asian_option(self.S,self.K,self.v,self.T,self.r-e,self.q)
        rho=(first_option.Asian_option_price(1000000,option_type,252,seed)-second_option.Asian_option_price(1000000,option_type,252,seed))/(2*e)
        return rho
    
    def Asian_option_gamma(self,e,option_type,seed=None):
        first_option=Asian_option(self.S+e,self.K,self.v,self.T,self.r,self.q)
        second_option=Asian_option(self.S-e,self.K,self.v,self.T,self.r,self.q)
        normal_option=Asian_option(self.S,self.K,self.v,self.T,self.r,self.q)
        gamma=(first_option.Asian_option_price(1000000,option_type,252,seed)-2*normal_option.Asian_option_price(1000000,option_type,252,seed)+second_option.Asian_option_price(1000000,option_type,252,seed))/(e**2)
        return gamma
    
    def summary(self,option_type,seed=None):
        if option_type.capitalize()=='Call' :
            print(f'the price of the Asian {option_type} option is {self.Asian_option_price(1000000,option_type,252,seed)} and the greeks are: Delta={self.Asian_option_delta(0.01,option_type,seed)}, Vega={self.Asian_option_vega(0.01,option_type,seed)}, Rho={self.Asian_option_rho(0.01,option_type,seed)}, Gamma={self.Asian_option_gamma(0.01,option_type,seed)}')
        elif option_type.capitalize()=='Put':
            print(f'the price of the Asian {option_type} option is {self.Asian_option_price(1000000,option_type,252,seed)} and the greeks are: Delta={self.Asian_option_delta(0.01,option_type,seed)}, Vega={self.Asian_option_vega(0.01,option_type,seed)}, Rho={self.Asian_option_rho(0.01,option_type,seed)}, Gamma={self.Asian_option_gamma(0.01,option_type,seed)}')
    

        
class Lookback_option(option):
    def Lookback_option_price(self,Number_of_simulations,option_type,steps,floating,seed=None):

        simulation=Brownian_simulator(self.S,self.v,self.T,self.r,self.q,Number_of_simulations,steps,seed)

        if option_type.capitalize()=='Call':
            if floating:
                lookback_payoff=np.maximum(simulation[:,-1]-np.min(simulation,axis=1),0)
                lookback_payoff=np.mean(lookback_payoff)*np.exp(-self.r*self.T)
                return lookback_payoff
            else:
                lookback_payoff=np.maximum(np.max(simulation,axis=1)-self.K,0)
                lookback_payoff=np.mean(lookback_payoff)*np.exp(-self.r*self.T)
                return lookback_payoff
        elif option_type.capitalize()=='Put':
            if floating:
                lookback_payoff=np.maximum(np.max(simulation,axis=1)-simulation[:,-1],0)
                lookback_payoff=np.mean(lookback_payoff)*np.exp(-self.r*self.T)
                return lookback_payoff
            else:
                lookback_payoff=np.maximum(self.K-np.min(simulation,axis=1),0)
                lookback_payoff=np.mean(lookback_payoff)*np.exp(-self.r*self.T)
                return lookback_payoff
        else:
            raise ValueError('please select Call or Put')


class Barrier_option(option):
    def Barrier_option_price(self,Number_of_simulations,option_type,barrier,barrier_condition,steps):

        simulation=Brownian_simulator(self.S,self.v,self.T,self.r,self.q,Number_of_simulations,steps,None)

        Final_price = simulation[:, -1]
        
        if option_type.capitalize() == 'Call':
            payoff_vanilla = np.maximum(Final_price - self.K, 0)
        elif option_type.capitalize() == 'Put':
            payoff_vanilla = np.maximum(self.K - Final_price, 0)
        
        if 'Up' in barrier_condition:
            has_touched = np.any(simulation > barrier, axis=1)
        elif 'Down' in barrier_condition:
            has_touched = np.any(simulation < barrier, axis=1)
        
        if 'In' in barrier_condition:
            final_payoff = payoff_vanilla * has_touched
        elif 'Out' in barrier_condition:
            final_payoff = payoff_vanilla * (~has_touched)
        
        return np.mean(final_payoff) * np.exp(-self.r * self.T)
                



In [7]:
option=Asian_option(100,100,0.2,1,0.05,0.02)
option.summary('call',42)

the price of the Asian call option is 5.161114611530302 and the greeks are: Delta=0.5503411484240051, Vega=0.21777832133694108, Rho=23.036876347741764, Gamma=0.03383317380034612
